# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [ ]:
#%pip install azure-ai-projects

In [1]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool
from dotenv import load_dotenv

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.13.4 (main, Jun  3 2025, 15:34:24) [Clang 17.0.0 (clang-1700.0.13.3)]'

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 09-Jun-2025 12:39:57


## 1. Grounding Bing Search

In [5]:
name = "Web search agent"
model = "gpt-4o-mini"

instructions = "You are an AI Agent that can do some web search using Bing"
connection_name = "aqbinggrounding001"

### Project client

In [6]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [7]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

In [8]:
# Create the agent with the Bing grounding tool
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-agent",
        instructions="You are a helpful agent",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_P9XmgZKtmeioNY3Rka4GZSRz


In [9]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_yT50stqtHOPyN91OYwwwJPaN


In [10]:
query = "What are the latest annoucements of Microsoft Build 2025?"

In [11]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_1UxmFesTueS57vYxMtdo1Q2y


In [12]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.FAILED
Run failed: {'code': 'tool_user_error', 'message': "Error: invalid_tool_input; The specified connection ID '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourcegroups/ai-foundry-rg/providers/microsoft.machinelearningservices/workspaces/aqmlworkspace/connections/aqbinggrounding001' was not found in the project or account connections. Please verify that the connection id in tool input is correct and exists in the project or account."}


In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

In [ ]:
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)

In [ ]:
query = "What is the current stock value for Microsoft?"

In [ ]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=query,
)
print(f"Created message, ID: {message.id}")

In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

In [ ]:
print("\033[1;31;34m")
messages = project_client.agents.list_messages(thread_id=thread.id)
print(messages.data[0].content[0].text.value)

In [ ]:
query = "What are the weather informations for Paris? Use emojis and be funny"

In [ ]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=query,
)
print(f"Created message, ID: {message.id}")

In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

In [ ]:
print("\033[1;31;34m")
messages = project_client.agents.list_messages(thread_id=thread.id)
print(messages.data[0].content[0].text.value)

### Post processing

In [ ]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    if agents.data[i].name == name:
        print(f"Deleting agent {agents.data[i].id}")
        project_client.agents.delete_agent(agents.data[i].id)